## Building a GPT

代码引用了[Karpathy的ng-video-lecture](https://github.com/karpathy/ng-video-lecture/tree/master)，我在note里面贡献了自己的注释理解，补充了相关理论。

In [1]:
# read it in to inspect it
with open('HarryPotterTxTFile.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  6435489


In [3]:
# let's look at the first 1000 characters
print(text[:1000])

Chapter : THE BOY WHO LIVED .
Mr and Mrs Dursley , of number four , Privet Drive , were proud to say that they were perfectly normal , thank you very much .
They were the last people you'd expect to be involved in anything strange or mysterious , because they just didn't hold with such nonsense .
Mr Dursley was the director of a firm called Grunnings , which made drills .
He was a big , beefy man with hardly any neck , although he did have a very large mustache .
Mrs Dursley was thin and blonde and had nearly twice the usual amount of neck , which came in very useful as she spent so much of her time craning over garden fences , spying on the neighbors .
The Dursley s had a small son called Dudley and in their opinion there was no finer boy anywhere .
The Dursleys had everything they wanted , but they also had a secret , and their greatest fear was that somebody would discover it .
They didn't think they could bear it if anyone found out about the Potters .
Mrs Potter was Mrs Dursley's 

**Jalen:**
<br />这里希望得到数据集中的字符集合chars, 以及集合chars的长度vocab_size, vocab_size将作为创建词向量映射哈希表的重要参数

In [4]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)



 !"&'(),.0123456789:?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
74


**Jalen:**
<br />这里encode()返回的是输入字符在基于字符集合chars创建的哈希表中的索引
<br />encode()则是求反解，由索引得到字符

In [5]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))


[55, 56, 56, 1, 67, 55, 52, 65, 52]
hii there


**Jalen:**
<br /> 将数据集所有字符encode()，并转化为tensor张量

In [6]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([6435489]) torch.int64
tensor([24, 55, 48, 63, 67, 52, 65,  1, 20,  1, 41, 29, 26,  1, 23, 36, 46,  1,
        44, 29, 36,  1, 33, 30, 43, 26, 25,  1,  9,  0, 34, 65,  1, 48, 61, 51,
         1, 34, 65, 66,  1, 25, 68, 65, 66, 59, 52, 72,  1,  8,  1, 62, 53,  1,
        61, 68, 60, 49, 52, 65,  1, 53, 62, 68, 65,  1,  8,  1, 37, 65, 56, 69,
        52, 67,  1, 25, 65, 56, 69, 52,  1,  8,  1, 70, 52, 65, 52,  1, 63, 65,
        62, 68, 51,  1, 67, 62,  1, 66, 48, 72,  1, 67, 55, 48, 67,  1, 67, 55,
        52, 72,  1, 70, 52, 65, 52,  1, 63, 52, 65, 53, 52, 50, 67, 59, 72,  1,
        61, 62, 65, 60, 48, 59,  1,  8,  1, 67, 55, 48, 61, 58,  1, 72, 62, 68,
         1, 69, 52, 65, 72,  1, 60, 68, 50, 55,  1,  9,  0, 41, 55, 52, 72,  1,
        70, 52, 65, 52,  1, 67, 55, 52,  1, 59, 48, 66, 67,  1, 63, 52, 62, 63,
        59, 52,  1, 72, 62, 68,  5, 51,  1, 52, 71, 63, 52, 50, 67,  1, 67, 62,
         1, 49, 52,  1, 56, 61, 69, 62, 59, 69, 52, 51,  1, 56, 61,  1, 48, 61,
      

In [7]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

**Jalen:**
<br />block_size表示希望模型处理的数据长度

In [8]:
block_size = 8
n_embd = 128
train_data[:block_size+1]

tensor([24, 55, 48, 63, 67, 52, 65,  1, 20])

**Jalen:**
 <br />NLP为一个顺序结果预测，当前位置的预测结果只与前面所有预测结果有关，
 <br />所以对于tensor([24, 55, 48, 63, 67, 52, 65,  1, 20])这一段数据来说，
 <br />当输入了24，则希望预测结果是55，
 <br />当55被成功预测，则[24, 55]参加下一个位置字符的预测，下一个希望预测结果是48,
 <br />当48被成功预测，则[24, 55, 48]参加下一个位置字符的预测，以此类推，直到达到设定预测字符最大长度
 <br />下面图片可视化了NLP的预测过程
 <br /><img src="../assert/LLM_predict_process.png" width="40%">

In [9]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([24]) the target: 55
when input is tensor([24, 55]) the target: 48
when input is tensor([24, 55, 48]) the target: 63
when input is tensor([24, 55, 48, 63]) the target: 67
when input is tensor([24, 55, 48, 63, 67]) the target: 52
when input is tensor([24, 55, 48, 63, 67, 52]) the target: 65
when input is tensor([24, 55, 48, 63, 67, 52, 65]) the target: 1
when input is tensor([24, 55, 48, 63, 67, 52, 65,  1]) the target: 20


**Jalen**
<br />x : 输入张量， y : 目标输出
<br />s.shape = [B, T], B : batch_size, GPU并行处理数据的纬度; T : seq_len, 数据的最大上下文长度
<br />这里的T理论上越大越好，但需要根据模型的建模能力来设置，因为Transformer的建模能力取决于MHA(Multi-head Attention)的头数，seq_len最长，意味着需要更多的MHA，也就意味着模型越复杂

In [10]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[53, 62, 72, 66,  1, 63, 65, 56],
        [56, 61, 67, 62,  1, 29, 48, 65],
        [ 1,  9,  0,  3,  1, 30,  1, 70],
        [51,  1, 62, 69, 52, 65,  1, 48]])
targets:
torch.Size([4, 8])
tensor([[62, 72, 66,  1, 63, 65, 56, 51],
        [61, 67, 62,  1, 29, 48, 65, 65],
        [ 9,  0,  3,  1, 30,  1, 70, 48],
        [ 1, 62, 69, 52, 65,  1, 48, 67]])
----
when input is [53] the target: 62
when input is [53, 62] the target: 72
when input is [53, 62, 72] the target: 66
when input is [53, 62, 72, 66] the target: 1
when input is [53, 62, 72, 66, 1] the target: 63
when input is [53, 62, 72, 66, 1, 63] the target: 65
when input is [53, 62, 72, 66, 1, 63, 65] the target: 56
when input is [53, 62, 72, 66, 1, 63, 65, 56] the target: 51
when input is [56] the target: 61
when input is [56, 61] the target: 67
when input is [56, 61, 67] the target: 62
when input is [56, 61, 67, 62] the target: 1
when input is [56, 61, 67, 62, 1] the target: 29
when input is [

In [11]:
print(xb) # our input to the transformer

tensor([[53, 62, 72, 66,  1, 63, 65, 56],
        [56, 61, 67, 62,  1, 29, 48, 65],
        [ 1,  9,  0,  3,  1, 30,  1, 70],
        [51,  1, 62, 69, 52, 65,  1, 48]])


**Jalen:**
> nn.Embedding( ) : 该模块通常用于存储单词嵌入并使用索引检索它们。模块的输入是索引列表，输出是相应的单词嵌入。

<br /> 我们需要使用嵌入层将将离散的字符表示为连续的向量表示。
<br />例如：

| Character | Index | Vector Representation |
|-----------|-------|----------------------|
|    a      |   0   |     (0.1, 0.2)       |
|    b      |   1   |     (0.3, 0.4)       |
|    c      |   2   |     (0.5, 0.6)       |
|   ...     |  ...  |        ...           |
|    z      |   25  |     (0.7, 0.8)       |
|  space    |   26  |     (0.9, 1.0)       |

<br>

> torch.multinomial( ) : 用于从多项分布中进行采样。多项分布是概率分布的一种，它描述了具有多个离散结果的试验或事件的概率分布情况。

<br />torch.multinomial函数的返回值是一个张量，其中包含了从给定概率分布中采样得到的离散结果的索引
<br />在generate中，我们执行max_new_tokens次，输入的第一个idx为prompt，generate会根据prompt来预测接下来的结果，i次的预测结果取决于i-1次的结果，直到预测结束
<br />在生成预测的过程中，我们只需要关注最后一步，即：**logits = logits[:, -1, :]**，因为logits的含义是前i步的所有结果，我们只需要输出第i步的结果，通过torch.multinomial预测出第i个字符索引，并添加到idx中

>**实际上transformer在tokenizer过程中，会在词嵌入向量上加上位置编码信息**


            def __init__(self, vocab_size):
                super().__init__()
                # each token directly reads off the logits for the next token from a lookup table
                self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
                self.position_embedding_table = nn.Embedding(block_size, n_embd)
                self.lm_head = nn.Linear(n_embd, vocab_size)

            def forward(self, idx, targets=None):
                B, T = idx.shape

                # idx and targets are both (B,T) tensor of integers
                tok_emb = self.token_embedding_table(idx)
                pos_emb = self.position_embedding_table(torch.arange(T))
                x = tok_emb + pos_emb
                logits = self.lm_head(x)


In [12]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 74])
tensor(4.7030, grad_fn=<NllLossBackward0>)

5Frngn4j
5xNA.jBB7h0HYEgpE.(qJyg!v9 NI&g.xBLR38"PNafM4Yzli3X)rFX5!mrii h5X6CP,C qjSTa.nDZ"T&)8 wHuxj


**均方误差（MSE）：** MSE是在回归问题中常用的损失函数，其目标是预测连续输出。它衡量了预测值与实际值之间的平均平方差，通常用于训练回归任务的神经网络。

**梯度下降（GD）：** 梯度下降是一种优化算法，用于最小化机器学习模型的损失函数。损失函数衡量了模型根据输入特征对目标变量的预测能力。梯度下降的思想是通过迭代地调整模型参数，沿着损失函数最陡峭的方向进行调整。

**动量（Momentum）：** 动量是随机梯度下降（SGD）的一种扩展，它在参数更新中添加了一个“动量”项。这个项有助于平滑更新，并使优化器能够继续朝着正确的方向移动，即使梯度改变方向或变化幅度。动量对于训练深度神经网络特别有用。

**RMSprop：** RMSprop是一种优化算法，它使用梯度的平方均值的移动平均来调整每个参数的学习率。这有助于避免参数更新的振荡，并在某些情况下改善收敛性。

**Adam：** Adam是一种流行的优化算法，结合了动量和RMSprop的思想。它使用梯度和梯度平方值的移动平均来调整每个参数的学习率。Adam通常被用作深度学习模型的默认优化器。

**AdamW：** AdamW是Adam优化器的一种修改版，它在参数更新中添加了权重衰减（weight decay）。这有助于对模型进行正则化，并可以提高泛化性能。

In [13]:
# create a PyTorch optimizer
optimizer = torch.optim.RMSprop(m.parameters(), lr=1e-3)

<br />SGD_Loss : 4.783581733703613
<br />RMSprop_Loss : 4.534679889678955
<br />Adam_Loss : 4.65895938873291
<br />AdamW_Loss : 4.65804386138916

In [14]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


4.534679889678955


In [15]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


ikf8d2XPKMyH4DtoUB 9y7II t
a6E2sbiSLh047DuevBDu5Lq6VC.o.xg.aI9"&FXe6.)YER
f1fa:a0zc8BP04?9B,PVH,EMi!sJ3Qbw?:nZNMq6CcvOY?jM:z&(dliePI4y!s8tdwS:zua:Un6OWXPFv&p.HpB6WvHfV

MZgqN0RzH8tdE Q0&E09
o'w(4E!s7oD x1kFrTnZ4lAihVB2fQS9bj1qejEMCxW,xZ4R0wI'tG:M?B!Vk'dAI8I03:uF
y75iSmz.nRDlQQRHuWLTxZNnc8OycC:Pf0nfF?nYy"8Pwh5LhYo7qJark!(v73 w)D5u7j3ZgXec9&7bODFy hED1"A"PN'ueJYYQmb&?mt!?i"fey97La:vqecK, k!v&C8kx.2vYFy,Aqnx"AePIgeZ4ifD,zj1yZ?6eZgd9V
gaHDZRHu33nX1zlnnhuOGWiSE1qmJ!tBCDX8Ng3 "AUNI LFSUvuB,zwHuHsJp"q7


## The mathematical trick in self-attention

In [16]:
# Jalen:
import time
class code_process_time:
    def start(self):
        self.start_time = time.time()
    def end(self):
        end_time = time.time()
        execution_time = end_time - self.start_time
        execution_time = end_time - self.start_time

        print("代码执行时间：", execution_time, "秒")

下面的代码中，展示了一种数学技巧 ———— 使用下三角矩阵，用叉乘方法计算b的前i行的均值 来代替 循环计算b的前i行的均值。这明显效率更高。
<br />在下面这里例子中，你可以看到，c和d的值是一样的，只不过他们用了不同的计算方式
<br />下面结果可以看到，第一种方法运算效率高于第二种

In [17]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
cal_time = code_process_time()
# Jalen:
# 使用下三角矩阵，用叉乘方法计算b的前i行的均值，并保存到c中
torch.manual_seed(42)

a = torch.tril(torch.ones(300, 300)) # 获取一个张量的下三角部分
print('tril_a=')
print(a)
print('--')
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(300,200)).float()
cal_time.start()
c = a @ b
cal_time.end()
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)
print('--')


# Jalen:
# 循环计算b的前i行的均值，并保存到d中
d = torch.zeros((b.shape[0],b.shape[1]))
cal_time.start()
for i in range(b.shape[0]):
    d[i] = (torch.mean(b[:i+1], dim=0))
cal_time.end()
print('d=')
print(d)

tril_a=
tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 1., 0., 0.],
        [1., 1., 1.,  ..., 1., 1., 0.],
        [1., 1., 1.,  ..., 1., 1., 1.]])
--
代码执行时间： 0.0009992122650146484 秒
a=
tensor([[1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0034, 0.0034, 0.0034,  ..., 0.0034, 0.0000, 0.0000],
        [0.0033, 0.0033, 0.0033,  ..., 0.0033, 0.0033, 0.0000],
        [0.0033, 0.0033, 0.0033,  ..., 0.0033, 0.0033, 0.0033]])
--
b=
tensor([[2., 7., 6.,  ..., 1., 6., 4.],
        [6., 9., 4.,  ..., 3., 9., 0.],
        [9., 5., 8.,  ..., 8., 7., 3.],
        ...,
        [9., 0., 6.,  ..., 7., 4., 3.],
        [9., 4., 2.,  ..., 5., 5., 5.],
        [1., 1., 3.,  ..., 9., 2., 4.]])
--
c=
tensor([[2.0000, 7.0000, 6.0000,  ..

**法一：循环计算权重累积**

In [18]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [19]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        
        xbow[b,t] = torch.mean(xprev, 0)


**法二：矩阵叉乘计算权重累积**

In [20]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

True

**法三：masked_fill + softmax**
<br />这里用masked_fill + softmax代替了法二的mean

In [21]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


True

**将法三用到self-attention的计算当中**
<br />还有值得说的一点是，这里将等于是mask(softmax(Q,K))，这样做的目的是因为nlp是线性预测任务，我们不希望前面的字符注意到没有预测出来的字符。
<br />这在LLAMA2的结构里也有同样的操作

> 下面的图为LLAMA2的结构，可以看到softmax(Q,K)也有一个mask操作
> 
<img src="../assert/LLAMA2.png" width="40%">

In [22]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
print(wei)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
         [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
         [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
         [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1687, 0.8313, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2477, 0.0514, 0.7008, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4410, 0.0957, 0.3747, 0.0887, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0069, 0.0456, 0.0300, 0.7748, 0.1427, 0.0000, 0.0000, 0.0000],
         [0.0660, 0.089

torch.Size([4, 8, 16])

In [23]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [24]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [25]:
k.var()

tensor(1.0449)

In [26]:
q.var()

tensor(1.0700)

In [27]:
wei.var()

tensor(1.0918)

In [28]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

这里展示了为什么Attention机制需要<code>wei = q @ k.transpose(-2, -1) * head_size**-0.5</code>,即除以一个d_k
<br />原因是softmax会锐化较大的值,可以看到上面的输出结果峰值比较平均，而下面的输出峰值明显
>Softmax公式如下：
$$
softmax(x_i) = \frac{e^{x_i}}{\sum^n_{i=0}e^{x_i}}
$$

In [29]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

下面的代码展示了Laynorm的构成细节
<br />并且这里和LLAMA2不同的是，LLAMA2使用的是RMSNorm
>LayerNorm公式如下：

$$
\hat{x}_i = \frac{x_i - \mu}{\sqrt{\sigma^2 + \epsilon}} \gamma + \beta

$$


In [30]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [31]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [32]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [33]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>



## Full finished code, for reference

You may want to refer directly to the git repo instead though.

**从下面的代码你可以看到，NanoGPT是一个decoder-only的结构**

In [34]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

with open('./HarryPotterTxTFile.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        # 残差链接
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.21089 M parameters
step 0: train loss 4.4421, val loss 4.4432
step 100: train loss 2.5018, val loss 2.4826
step 200: train loss 2.3833, val loss 2.3681
step 300: train loss 2.3010, val loss 2.2799
step 400: train loss 2.2375, val loss 2.2134
step 500: train loss 2.1652, val loss 2.1483
step 600: train loss 2.1038, val loss 2.0803
step 700: train loss 2.0476, val loss 2.0402
step 800: train loss 2.0060, val loss 2.0060
step 900: train loss 1.9575, val loss 1.9550
step 1000: train loss 1.9264, val loss 1.9143
step 1100: train loss 1.8898, val loss 1.8812
step 1200: train loss 1.8629, val loss 1.8555
step 1300: train loss 1.8384, val loss 1.8170
step 1400: train loss 1.8093, val loss 1.8080
step 1500: train loss 1.7966, val loss 1.8017
step 1600: train loss 1.7681, val loss 1.7757
step 1700: train loss 1.7696, val loss 1.7655
step 1800: train loss 1.7416, val loss 1.7554
step 1900: train loss 1.7344, val loss 1.7307
step 2000: train loss 1.7111, val loss 1.7221
step 2100: train loss 1.6